### Code to create monthly sum,avg

In [1]:
%env CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt

env: CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt


In [2]:
# import sys
import datetime
import os
# import time
import rasterio
import numpy as np
# from glob import glob

In [3]:

def create_s3_list_of_days(main_prefix, year, output_name='etasw_'):
    the_list = []
    for i in range(1,366):
        month = f'{i:03d}'
        file_object = main_prefix + '/' + str(year) + '/' + output_name + str(year) + month + '.tif'
        the_list.append(file_object)
    return the_list


In [4]:
working_bucket = 'dev-et-data'
main_prefix = 's3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer'
year = 2003
#output_name = 'etasw_'
#output_name = 'dd_'
output_name = 'srf_'


ppt_list = create_s3_list_of_days(main_prefix, year, output_name)
ppt_list 



['s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003001.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003002.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003003.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003004.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003005.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003006.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003007.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003008.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003009.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/

In [5]:
#read in file with rasterio
def read_file(file):
    print("reading file ...", file)
    with rasterio.open(file) as src:
        return(src.read(1))

In [6]:
def monthly_average(file_list, out_dir, out_product):
# what months to summarize
    start_mon = 1 #start month
    end_mon = 12 #end month


    #loop through month 1,2,..12    
    for i in range(start_mon,(end_mon+1)): 
        print('Month averaged up is: ' + str(i))
        Listras = [] 
        for et_in in file_list:
            doy = int(et_in.split('.')[0][-3:])
            #doy = int(et_in[-3:])
            #print 'Day of the year: ' + str(doy)
            datea = str(datetime.date(year,1,1) + datetime.timedelta(doy-1))
            mon = int(datea.split('-')[1])
            #print 'Month is: ' + str(mon)
            if mon == i: #if month = i then append grid to list for summing up
                Listras.append(et_in)
        #print('daily grids for month ' + str(i) + ' :')
        #print(Listras)
        if Listras == []:
            print('No daily data for month' + str(i) + ' available..continue to next month')
            continue
        else:
            # Read all data as a list of numpy arrays 
            array_list = [read_file(x) for x in Listras]
            # Perform averaging
            array_out = np.nanmean(array_list, axis=0)

            # Get metadata from one of the input files
            with rasterio.open(file_list[0]) as src:
                meta = src.meta
            meta.update(dtype=rasterio.float32)

            # Write output file
            #out_name = 'ppt_avg_' + str(year) + (('0'+ str(i))[-2:]) +'.tif'
            out_name = out_product + str(year) + (('0'+ str(i))[-2:]) +'.tif'

            with rasterio.open(out_dir + '/' + out_name, 'w', **meta) as dst:
                dst.write(array_out.astype(rasterio.float32), 1)

            print('Created monthly grid!', out_name)


In [7]:
def _mkdir(directory):
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)

In [8]:
_mkdir('./junkbox')

In [9]:

out_dir = './junkbox'
out_product = 'srf_avg_'
for year in range(2003,2016):
    file_list = create_s3_list_of_days(main_prefix, year, output_name)
    monthly_average(file_list, out_dir, out_product)

Month averaged up is: 1
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003001.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003002.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003003.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003004.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003005.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003006.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003007.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003008.tif
reading file ... s3://de

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: Mean of empty slice


Created monthly grid! srf_avg_200301.tif
Month averaged up is: 2
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003032.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003033.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003034.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003035.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003036.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003037.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003038.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003

reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003100.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003101.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003102.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003103.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003104.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003105.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003106.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003107.tif
reading file ... s3://dev-et-data/enduser/Delawa

reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003168.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003169.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003170.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003171.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003172.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003173.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003174.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2003/srf_2003175.tif
reading file ... s3://dev-et-data/enduser/Delawa

...

reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015072.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015073.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015074.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015075.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015076.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015077.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015078.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015079.tif
reading file ... s3://dev-et-data/enduser/Delawa

reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015140.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015141.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015142.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015143.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015144.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015145.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015146.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015147.tif
reading file ... s3://dev-et-data/enduser/Delawa

reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015208.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015209.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015210.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015211.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015212.tif
Created monthly grid! srf_avg_201507.tif
Month averaged up is: 8
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015213.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015214.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015

reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015276.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015277.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015278.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015279.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015280.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015281.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015282.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015283.tif
reading file ... s3://dev-et-data/enduser/Delawa

reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015344.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015345.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015346.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015347.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015348.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015349.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015350.tif
reading file ... s3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/2015/srf_2015351.tif
reading file ... s3://dev-et-data/enduser/Delawa

In [10]:
! ls -lh ./junkbox

total 20G
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 43M Oct  7 01:38 dd_avg_200301.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 43M Oct  7 01:38 dd_avg_200302.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 43M Oct  7 01:40 dd_avg_200303.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 43M Oct  7 01:40 dd_avg_200304.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 43M Oct  7 01:41 dd_avg_200305.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 43M Oct  7 01:41 dd_avg_200306.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 43M Oct  7 01:42 dd_avg_200307.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 43M Oct  7 01:42 dd_avg_200308.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 43M Oct  7 01:43 dd_avg_200309.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 43M Oct  7 01:44 dd_avg_200310.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 43M Oct  7 01:44 dd_avg_200311.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 43M Oct  7 01:45 dd_avg_200312.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 43M Oct  7 01:46 dd_avg_200401.tif
-r